In [ ]:
#!/usr/bin/env python
# coding: utf-8


In [ ]:
# In[2]:


In [ ]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import random
import os


In [ ]:

# In[3]:


In [ ]:

def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)


In [ ]:
SEED = 42
fix_seed(SEED)


In [ ]:

# ESサイトのユーザーに対して**RCT**を適用したメールマーケティングを行ったデータ。


In [ ]:
# In[5]:


In [ ]:

# データのロード
base_path = "/Users/ryoto/workspace/hit-u/zemi/econome_ml_demo/data"
mail = pd.read_csv(os.path.join(base_path,'E-MailAnalytics.csv'))
print(mail.shape)
mail.head()


In [ ]:

# In[6]:


In [ ]:

print("num of No E-Mail : ", len(mail.query("segment == 'No E-Mail'")))
print("num of Mens E-Mail : ", len(mail.query("segment == 'Womens E-Mail'")))
print("num of No Womens : ", len(mail.query("segment == 'Mens E-Mail'")))


In [ ]:

# ## RCTのデータからATEを求めてBaseLineとする
# 簡略化のために女性向けのメールが配信されているデータを削除する。


In [ ]:
# In[7]:


In [ ]:

mail_df = mail.query("segment != 'Womens E-Mail'")
print(len(mail_df.query("segment == 'No E-Mail'")))
print(len(mail_df.query("segment == 'Womens E-Mail'")))
print(len(mail_df.query("segment == 'Mens E-Mail'")))


In [ ]:

# In[8]:


In [ ]:

mail_df["segment"] = mail_df.segment.map(lambda x: 1 if x == 'Mens E-Mail' else 0)
mail_df = mail_df.rename(columns={"segment" : "treatment"})
mail_df.head()


In [ ]:

# In[9]:


In [ ]:

mail_df.groupby("treatment").agg({"spend" : "mean", "conversion": "mean", "visit":"count"}).rename(columns={"visit" : "count"})


In [ ]:

# In[10]:


In [ ]:

# treatment 0, 1 のデータフレームに分ける
treatment_1 = mail_df.query("treatment == 1")
treatment_0 = mail_df.query("treatment == 0")


In [ ]:
# 介入が購買金額に与えた影響を計算する
ts_1 = treatment_1["spend"].mean()
ts_0 = treatment_0["spend"].mean()
print("介入が購買金額に与えた影響は,", np.round(ts_1 - ts_0, 3))


In [ ]:
# 介入がconversionに与えた影響を計算する
tc_1 = treatment_1["conversion"].mean()
tc_0 = treatment_0["conversion"].mean()
True_ATE = np.round((tc_1 - tc_0), 5)
print("介入がconversionに与えた影響は,", True_ATE)


In [ ]:

# ## データの前処理


In [ ]:
# In[37]:


In [ ]:

mail_df.head()


In [ ]:

# In[38]:


In [ ]:

# ラベルエンコーディング（OrdinalEncoder）
from sklearn.preprocessing import OrdinalEncoder


In [ ]:
oe = OrdinalEncoder()
encoded = oe.fit_transform(mail_df[['history_segment', 'zip_code', "channel"]].values)
# decoded = oe.inverse_transform(encoded)


In [ ]:
print('エンコード結果: ')
encoded_df = pd.DataFrame(encoded, columns = ["history_segment", "zip_code", "channel"])
encoded_df.head()


In [ ]:

# In[39]:


In [ ]:

mail_df_use = mail_df[
    ['recency', 'history', 'mens', 'womens', 'newbie', 'treatment', 'visit', 'conversion', 'spend']
    ]
demo_df = pd.concat([mail_df_use.reset_index(), encoded_df], axis = 1).drop(columns = "index")
print(demo_df.shape)
demo_df.head()


In [ ]:

# In[40]:


In [ ]:

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(demo_df, test_size= 0.2,random_state=SEED)
print(train_df.shape)
print(val_df.shape)


In [ ]:

# ## Econmlで試す


In [ ]:
# In[41]:


In [ ]:

# Main imports
from econml.metalearners import TLearner, SLearner, XLearner, DomainAdaptationLearner


In [ ]:
# Helper imports 
import numpy as np
from numpy.random import binomial, multivariate_normal, normal, uniform
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
import matplotlib.pyplot as plt


In [ ]:

# In[42]:


In [ ]:

# set data 
Y = train_df.conversion
T = train_df.treatment
X = train_df.drop(columns=["conversion", "treatment", "spend", "visit"])
X_val = val_df.drop(columns=["conversion", "treatment", "spend", "visit"])


In [ ]:

# ### T-Learner


In [ ]:
# In[43]:


In [ ]:

# Instantiate T learner
models = RandomForestRegressor(n_estimators=1000, max_depth=10, min_samples_leaf=64, random_state = 42)
T_learner = TLearner(models=models)
# Train T_learner
T_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
T_te = T_learner.effect(X_val)


In [ ]:
print("True ATE : ", True_ATE)
pred_ATE_T = np.round(T_learner.ate(X_val), 5)
print("Predict ATE : ", pred_ATE_T)


In [ ]:

# ### S-Learner


In [ ]:
# In[44]:


In [ ]:

# Instantiate S learner
overall_model = RandomForestRegressor(n_estimators=1000, max_depth=6, min_samples_leaf=64, random_state = 42)
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
S_te = S_learner.effect(X_val)


In [ ]:
print("True ATE : ", True_ATE)
pred_ATE_S = np.round(S_learner.ate(X_val), 5)
print("Predict ATE : ", pred_ATE_S)


In [ ]:

# ### X-Learner


In [ ]:
# In[45]:


In [ ]:

# Instantiate X learner
models = RandomForestRegressor(n_estimators=1500, max_depth= 10 , min_samples_leaf=64, random_state = 42)
propensity_model = RandomForestClassifier(n_estimators=500, max_depth = 10 , min_samples_leaf=64, random_state = 42)
X_learner = XLearner(models=models, propensity_model=propensity_model)
# Train X_learner
X_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
X_te = X_learner.effect(X_val)
print("True ATE : ", True_ATE)
pred_ATE_X = np.round(X_learner.ate(X_val), 5)
print("Predict ATE : ", pred_ATE_X)


In [ ]:

# # Use_fulldata


In [ ]:
# In[46]:


In [ ]:

# set data 
Y = demo_df.conversion
T = demo_df.treatment
X = demo_df.drop(columns=["conversion", "treatment", "spend", "visit"])
print(demo_df.shape)


In [ ]:

# ## T-Learner


In [ ]:
# In[47]:


In [ ]:

# Instantiate T learner
models = RandomForestRegressor(n_estimators=1000, max_depth=10,  random_state = 42)
T_learner = TLearner(models=models)
# Train T_learner
T_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
T_te = T_learner.effect(X)


In [ ]:
print("True ATE : ", True_ATE)
pred_ATE_T = np.round(T_learner.ate(X), 6)
print("Predict ATE : ", pred_ATE_T)


In [ ]:

# In[48]:


In [ ]:

# Instantiate T learner
models = GradientBoostingRegressor(n_estimators=1000, max_depth=10,  random_state = 42)
T_learner = TLearner(models=models)
# Train T_learner
T_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
T_te = T_learner.effect(X)


In [ ]:
print("True ATE : ", True_ATE)
pred_ATE_T = np.round(T_learner.ate(X), 6)
print("Predict ATE : ", pred_ATE_T)


In [ ]:

# ## S-Learner


In [ ]:
# In[49]:


In [ ]:

# Instantiate S learner
overall_model = RandomForestRegressor(n_estimators=1000, max_depth=10,  random_state = 42)
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
S_te = S_learner.effect(X)


In [ ]:
print("True ATE : ", True_ATE)
pred_ATE_S = np.round(S_learner.ate(X), 5)
print("Predict ATE : ", pred_ATE_S)


In [ ]:

# In[50]:


In [ ]:

# Instantiate S learner
overall_model = GradientBoostingRegressor(n_estimators=1000, max_depth=10,  random_state = 42)
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
S_te = S_learner.effect(X)


In [ ]:
print("True ATE : ", True_ATE)
pred_ATE_S = np.round(S_learner.ate(X), 5)
print("Predict ATE : ", pred_ATE_S)


In [ ]:

# ## X-Learner


In [ ]:
# In[51]:


In [ ]:

# Instantiate X learner
models = RandomForestRegressor(n_estimators=1000, max_depth=12,  random_state = 42)
propensity_model = RandomForestClassifier(n_estimators=1000, max_depth=12,  random_state = 42)
X_learner = XLearner(models=models, propensity_model=propensity_model)
# Train X_learner
X_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
X_te = X_learner.effect(X)
print("True ATE : ", True_ATE)
pred_ATE_X = np.round(X_learner.ate(X), 5)
print("Predict ATE : ", pred_ATE_X)


In [ ]:

# In[52]:


In [ ]:

# Instantiate X learner
models = GradientBoostingRegressor(n_estimators=1000, max_depth=12,  random_state = 42)
propensity_model = RandomForestClassifier(n_estimators=1000, max_depth=12,  random_state = 42)
X_learner = XLearner(models=models, propensity_model=propensity_model)
# Train X_learner
X_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
X_te = X_learner.effect(X)
print("True ATE : ", True_ATE)
pred_ATE_X = np.round(X_learner.ate(X), 5)
print("Predict ATE : ", pred_ATE_X)
